
## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Source #Mobile

### Links: 
___

## Creating a table from s3

doesn't work now

In [ ]:
query_text = """--sql
    CREATE TABLE db1.citizens_st_mobile_parquet
    (
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `activated_at` String,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_parquet/year=*/month=*/*.parquet','parquet')
    """

ch.query_run(query_text)

## Creating table in CH

In [ ]:
query_text = """--sql
    CREATE TABLE db1.citizens_st_mobile_parquet_ch
    (
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `activated_at` String,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """

ch.query_run(query_text)

## Creating MV

doesn't work now

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.citizens_st_mobile_parquet_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR 30 MINUTE RANDOMIZE FOR 1 HOUR TO db1.citizens_st_mobile_parquet_ch AS
    SELECT
        *
    FROM db1.citizens_st_mobile_parquet
    """

ch.query_run(query_text)

___
## Tools

### query

In [ ]:
query_text = """--sql
    SELECT
        *
    FROM db1.citizens_st_mobile_parquet_ch
    WHERE report_date = '2025-05-29'
    limit 100
    """

ch.query_run(query_text)

### Drop ch

In [3]:
query_text = """
    DROP TABLE db1.citizens_st_mobile_parquet_ch
    """
ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 359
 Code: 359. DB::Exception: Table or Partition in db1.citizens_st_mobile_parquet_ch was not dropped.
Reason:
1. Size (65.84 GB) is greater than max_[table/partition]_size_to_drop (53.69 GB)
2. File '/var/lib/clickhouse/flags/force_drop_table' intended to force DROP doesn't exist
How to fix this:
1. Either increase (or set to zero) max_[table/partition]_size_to_drop in server config
2. Either pass a bigger (or set to zero) max_[table/partition]_size_to_drop through query settings
3. Either create forcing file /var/lib/clickhouse/flags/force_drop_table and make sure that ClickHouse has write permission for it.
Example:
sudo touch '/var/lib/clickhouse/flags/force_drop_table' && sudo chmod 666 '/var/lib/clickhouse/flags/force_drop_table'. (TABLE_SIZE_EXCEEDS_MAX_DROP_SIZE_LIMIT) (version 25.3.3.42 (official build))


### Drop mv

In [4]:
query_text = """
    DROP TABLE db1.citizens_st_mobile_parquet_mv
    """
ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Table db1.citizens_st_mobile_parquet_mv does not exist. Maybe you meant db1.citizens_st_mobile_parquet_mv?. (UNKNOWN_TABLE) (version 25.3.3.42 (official build))
